# Analytics

## Imports

In [1]:
import sys

# Add the parent directory to the Python path to access Scrapy files
sys.path.append('../')

import pymongo
from scraper import settings
from scrapy.crawler import Crawler
from scraper.pipelines import ScraperPipeline
import pandas as pd
import re

## Connect to the MongoDB database

In [2]:
# Access the MongoDB connection information from the settings
mongo_uri = settings.MONGO_URI
mongo_port = settings.MONGO_PORT
mongo_db = settings.MONGO_DATABASE

# Create a MongoDB client
client = pymongo.MongoClient(mongo_uri, mongo_port)

# Access the database
db = client[mongo_db]

# Access the collection
watches = db[ScraperPipeline.collection_name]

In [3]:
examples = watches.find().limit(3)
for example in examples:
    print(example)

{'_id': ObjectId('6547629977ddcb0f219c67da'), 'image_urls': ['https://chronexttime.imgix.net/V/6/V61966/V61966_1_det.png?w=1000&ar=1:1&auto=format&fm=png&q=55&usm=50&usmrad=1.5&dpr=1&trim=color&fit=fill&auto=compress&bg=FFFFFF&bg-remove=true', 'https://chronexttime.imgix.net/V/6/V61966/V61966_1.png?w=1000&ar=1:1&auto=format&fm=png&q=55&usm=50&usmrad=1.5&dpr=1&trim=color&fit=fill&auto=compress&bg=FFFFFF&bg-remove=false', 'https://chronexttime.imgix.net/V/6/V61966/V61966_2.jpg?w=1000&ar=1:1&auto=format&fm=jpg&q=55&usm=50&usmrad=1.5&dpr=1&trim=color&fit=fill&auto=compress&bg=FFFFFF&bg-remove=false', 'https://chronexttime.imgix.net/V/6/V61966/V61966_3.jpg?w=1000&ar=1:1&auto=format&fm=jpg&q=55&usm=50&usmrad=1.5&dpr=1&trim=color&fit=fill&auto=compress&bg=FFFFFF&bg-remove=false', 'https://chronexttime.imgix.net/V/6/V61966/V61966_4.jpg?w=1000&ar=1:1&auto=format&fm=jpg&q=55&usm=50&usmrad=1.5&dpr=1&trim=color&fit=fill&auto=compress&bg=FFFFFF&bg-remove=false'], 'image_paths': ['full/3f90e8c550279

In [4]:
# Count the number of documents in the collection
print(f"Number of items in the collection: {watches.count_documents({})}")

Number of items in the collection: 1482


## Descriptive statistics

In [5]:
raw_data = pd.json_normalize(list(watches.find({})))
raw_data.head(3)

,_id,image_urls,image_paths,thumb_paths,état,année,marque,modèle,référence,mouvement,dimensions,genre,sku,boîtier,bracelet,cristal,couleur du cadran,fonctions,price
0,6547629977ddcb0f219c67da,[https://chronexttime.imgix.net/V/6/V61966/V61...,[full/3f90e8c550279709a1bd0e96de95831a3642cdd4...,[thumbs/small/3f90e8c550279709a1bd0e96de95831a...,non-portée - neuve,2023,omega,seamaster diver 300m,210.32.42.20.04.001,automatique,42mm,homme,v61966,acier inoxydable,caoutchouc,saphir,blanc,"[date, valve à hélium, aiguilles lumineuses, i...",5300.0
1,6547629c77ddcb0f219c67db,[https://chronexttime.imgix.net/V/6/V67443/V67...,[full/2fb3346adeb4ed7cf90b93b8e9c9052d5deaaa5b...,[thumbs/small/2fb3346adeb4ed7cf90b93b8e9c9052d...,non-portée - neuve,2023,breitling,avenger chronograph 45 night mission,v13317101l1x2,automatique,45mm,homme,v67443,titane,cuir,saphir,vert,"[aiguilles lumineuses, indice de luminosité, l...",4550.0
2,6547629f77ddcb0f219c67dc,[https://chronexttime.imgix.net/V/6/V67506/V67...,[full/1137d34ac9f8277bf4332e62a204f6ee723023ad...,[thumbs/small/1137d34ac9f8277bf4332e62a204f6ee...,certifiée d'occasion - bien,2020,tag heuer,carrera,cbn2a10.ba0643,automatique,44mm,homme,v67506,acier inoxydable,acier inoxydable,saphir,vert,"[date, couronne vissée, petite seconde, tachym...",4870.0


In [6]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482 entries, 0 to 1481
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   _id                1482 non-null   object 
 1   image_urls         1482 non-null   object 
 2   image_paths        1482 non-null   object 
 3   thumb_paths        1482 non-null   object 
 4   état               1482 non-null   object 
 5   année              1419 non-null   object 
 6   marque             1482 non-null   object 
 7   modèle             1482 non-null   object 
 8   référence          1482 non-null   object 
 9   mouvement          1439 non-null   object 
 10  dimensions         1461 non-null   object 
 11  genre              1482 non-null   object 
 12  sku                1482 non-null   object 
 13  boîtier            1469 non-null   object 
 14  bracelet           1467 non-null   object 
 15  cristal            1445 non-null   object 
 16  couleur du cadran  1459 

In [7]:
raw_data.isna().sum()

_id                    0
image_urls             0
image_paths            0
thumb_paths            0
état                   0
année                 63
marque                 0
modèle                 0
référence              0
mouvement             43
dimensions            21
genre                  0
sku                    0
boîtier               13
bracelet              15
cristal               37
couleur du cadran     23
fonctions            147
price                  0
dtype: int64

In [8]:
raw_data.describe(include='object')

,_id,image_urls,image_paths,thumb_paths,état,année,marque,modèle,référence,mouvement,dimensions,genre,sku,boîtier,bracelet,cristal,couleur du cadran,fonctions
count,1482,1482,1482,1482,1482,1419,1482,1482,1482,1439,1461,1482,1482,1469,1467,1445,1459,1335
unique,1482,1482,1482,1482,5,61,69,706,1252,3,74,4,1482,31,39,5,19,467
top,6547629977ddcb0f219c67da,[https://chronexttime.imgix.net/V/6/V61966/V61...,[full/3f90e8c550279709a1bd0e96de95831a3642cdd4...,[thumbs/small/3f90e8c550279709a1bd0e96de95831a...,non-portée - neuve,2023,rolex,datejust 41,126300,automatique,41mm,homme,v61966,acier inoxydable,acier inoxydable,saphir,noir,"[date, minute, heure, seconde au centre]"
freq,1,1,1,1,808,723,290,51,20,1172,213,929,1,928,571,1393,518,59


In [9]:
raw_data.describe()

,price
count,1482.000000
mean,11823.130904
std,18934.153677
min,800.000000
25%,2840.000000
50%,6080.000000
75%,12537.500000
max,239000.000000


In [10]:
# Summarize each categorical variable
def summarize(df, cols, cols_to_explode=None):
    for col in cols:
        if cols_to_explode and col in cols_to_explode:
            print(df.explode(col)[col].value_counts(dropna=False).to_string())
        else:
            print(df[col].value_counts(dropna=False).to_string())
        print()

In [11]:
cols_to_summarize = ["état", "mouvement", "dimensions", "genre", "boîtier", "bracelet", "cristal", "couleur du cadran"]
summarize(raw_data, cols_to_summarize)

état
non-portée - neuve                    808
certifiée d'occasion - très bien      264
certifiée d'occasion - comme neuve    253
certifiée d'occasion - bien           139
certifiée d'occasion - vintage         18

mouvement
automatique         1172
quartz               172
remontage manuel      95
NaN                   43

dimensions
41mm       213
40mm       211
42mm       173
36mm       124
44mm        91
39mm        90
43mm        86
38mm        66
34mm        42
45mm        36
37mm        30
33mm        28
NaN         21
32mm        20
28mm        19
35mm        19
31mm        18
30mm        16
46mm        15
29mm        11
43.5mm      11
38.5mm      10
39.5mm      10
45.5mm       8
42.5mm       7
25mm         7
26mm         7
48mm         6
23mm         6
44.25mm      6
24mm         5
43.8mm       4
22mm         4
19mm         3
47mm         3
27mm         3
36.6mm       2
44.9mm       2
29.5mm       2
50mm         2
36.3mm       2
22.7mm       2
45.8mm       2
38.3mm       2
35

In [45]:
# Preprocess
# TODO : apply good types
# TODO : flag 'plot', 'text', 'both'

def preprocess(df):
    new_df = df.copy(deep=True)
    
    new_df["état"] = new_df["état"].apply(lambda x: x.split("-")[-1].strip())
    new_df["genre"] = new_df["genre"].replace("unisex", "unisexe")
    new_df["fonctions"] = new_df["fonctions"].fillna("[]")
    new_df["n_fonctions"] = new_df["fonctions"].apply(lambda x: len(x))
    new_df.replace("", "inconnu", inplace=True)
    new_df.fillna("inconnu", inplace=True)
    new_df.drop(columns=["image_urls"], inplace=True)

    def split(s):
        return [elem.strip() for elem in s.split("/")]

    new_df["boîtier"] = new_df["boîtier"].apply(split)
    new_df["bracelet"] = new_df["bracelet"].apply(split)

    trim = re.compile(r"[^\d.,]+")  

    def get_interval(s):
        if s == "inconnu":
            return s
        
        num = float(trim.sub("", s).replace(",", "."))

        if num < 15:
            return "< 15"
        elif 15 <= num and num < 20:
            return "[15, 20["
        elif 20 <= num and num < 25:
            return "[20, 25["
        elif 25 <= num and num < 30:
            return "[25, 30["
        elif 30 <= num and num < 35:
            return "[30, 35["  
        elif 35 <= num and num < 40:
            return "[35, 40["  
        elif 40 <= num and num < 45:
            return "[40, 45["
        elif 45 <= num and num < 50:
            return "[45, 50["  
        else:
            return ">= 50"
    
    new_df["dimensions"] = new_df["dimensions"].apply(get_interval)

    # Explode dataframe images
    new_df = df.explode(["image_paths", "thumb_paths"])

    return new_df

In [13]:
images = preprocess(raw_data)
images.head(3)

In [ ]:
images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482 entries, 0 to 1481
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   _id                1482 non-null   object 
 1   image_urls         1482 non-null   object 
 2   image_paths        1482 non-null   object 
 3   thumb_paths        1482 non-null   object 
 4   état               1482 non-null   object 
 5   année              1419 non-null   object 
 6   marque             1482 non-null   object 
 7   modèle             1482 non-null   object 
 8   référence          1482 non-null   object 
 9   mouvement          1439 non-null   object 
 10  dimensions         1461 non-null   object 
 11  genre              1482 non-null   object 
 12  sku                1482 non-null   object 
 13  boîtier            1469 non-null   object 
 14  bracelet           1467 non-null   object 
 15  cristal            1445 non-null   object 
 16  couleur du cadran  1459 

In [ ]:
images.isna().sum()

_id                    0
image_urls             0
image_paths            0
thumb_paths            0
état                   0
année                 63
marque                 0
modèle                 0
référence              0
mouvement             43
dimensions            21
genre                  0
sku                    0
boîtier               13
bracelet              15
cristal               37
couleur du cadran     23
fonctions            147
price                  0
dtype: int64

In [ ]:
images.describe(include='object')

,_id,image_urls,image_paths,thumb_paths,état,année,marque,modèle,référence,mouvement,dimensions,genre,sku,boîtier,bracelet,cristal,couleur du cadran,fonctions
count,1482,1482,1482,1482,1482,1419,1482,1482,1482,1439,1461,1482,1482,1469,1467,1445,1459,1335
unique,1482,1482,1482,1482,5,61,69,706,1252,3,74,4,1482,31,39,5,19,467
top,6547629977ddcb0f219c67da,[https://chronexttime.imgix.net/V/6/V61966/V61...,[full/3f90e8c550279709a1bd0e96de95831a3642cdd4...,[thumbs/small/3f90e8c550279709a1bd0e96de95831a...,non-portée - neuve,2023,rolex,datejust 41,126300,automatique,41mm,homme,v61966,acier inoxydable,acier inoxydable,saphir,noir,"[date, minute, heure, seconde au centre]"
freq,1,1,1,1,808,723,290,51,20,1172,213,929,1,928,571,1393,518,59


In [ ]:
images.describe()

,price
count,1482.000000
mean,11823.130904
std,18934.153677
min,800.000000
25%,2840.000000
50%,6080.000000
75%,12537.500000
max,239000.000000


In [14]:
cols_to_summarize = ["état", "mouvement", "dimensions", "genre", "boîtier", "bracelet", "cristal", "couleur du cadran", "n_fonctions"]
cols_to_explode = ["boîtier", "bracelet"]

summarize(images, cols_to_summarize, cols_to_explode)

état
neuve          808
très bien      264
comme neuve    253
bien           139
vintage         18

mouvement
automatique         1172
quartz               172
remontage manuel      95
inconnu               43

dimensions
[40, 45[    816
[35, 40[    363
[30, 35[    129
[45, 50[     73
[25, 30[     54
inconnu      21
[20, 25[     20
[15, 20[      4
>= 50         2

genre
homme      929
unisexe    296
femme      257

boîtier
acier inoxydable                   1090
or jaune                            158
or rose                              95
céramique                            79
or blanc                             70
titane                               66
or rouge                             16
inconnu                              16
bronze                               16
platine                              14
acier plaqué or                       7
argent                                6
acier avec revêtement pvd jaune       5
carbone                               4
or          

In [16]:
# Most important for image generation : statistics about the images we collected

In [ ]:
# Number of images per brand
# Too much brand
# Check if very frequent brands -> 10 brands + others

In [ ]:
# Average image count per item
count = images.groupby("_id").size().mean().astype(int)
print(f"Average image count per item: {count}")


In [ ]:
# Average image count per item per brand
count = images.groupby(["marque", "_id"]).size().groupby("marque").mean().astype(int)
print(f"Average image count per item per brand: {count}")

In [ ]:
# TODO :

# Item count per brand

# Number of distinct models per brand

# Average price per brand

# Frequency of watch functions. Most common functions

# Average price per number of functions

In [ ]:
# TODO : plots
# Can create plots in the summarize function


# For price : Boxplot total, Boxplot per brand, Boxplot per number of functions
# Rq: stats sur les prix pas vraiment intéressantes. Objectif principal = voir diversité des montres dans le jeu construit.